# Optymalizacja - wstep

## Przygotowanie bibliotek

In [11]:
import numpy as np
import cvxpy as cp
import warnings
warnings.filterwarnings('ignore')

## Teoria

Zadania optymalizacyjne to takie działania, które mają na celu znalezienie najlepszego rozwiązania określonego problemu. 

Na każdy problem optymalizacyjny składają się co najmniej trzy elementy:

1. **Definicja funkcji celu** - co ma być optymalizowane
2. **Definicja zmiennych** - jakimi czynnikami mozna manipulować aby ten cel osiągnąć
3. **Definicja ograniczeń** - w jaki sposób można modyfikować zmienne

**Minimalizacja / optymalizacja**

Klasyczne roblemy optymalizacyjne przedstawia się jako *minimalizację* określonej funkcji.

Nic nie stoi jednak na przeszkodzie, aby minimalizację zamienić w maksymalizację:

$$ Minimalizacja_x\ f(x) \Leftrightarrow Maksymalizacja_x\ f(-x) $$

**Typy problemów**

W zależności od rodzaju zmiennych uwzględnianych w problemie optymaliacyjnym, można wyróżnić następujące ich typy:

1. **Problemy binarne** - gdzie zmienne przyjmują wartości 0 lub 1
2. **Problemy całkowitoliczbowe** - gdzie zmienne mogą przyjmować wartości będące tylko liczbami całkowitymi
3. **Problemy mieszane** - połączenie powyższych

**Formalizacja**

Problemy optymalizacyjne zazwyczaj przedstawiane są w postaci macierzowej, gdzie określa się:

1. **x** - macierz zmiennych. Czynniki, którymi można manipuloawć
2. **A** - macierz współczynników przypisywanych do zmiennych
3. b - wektor ograniczeń dla każdej ze zmiennych
4. c - wektor odnoszący się do wartości funkcji celu, przypisywany do zmiennych

Mając takie komponenty, można stworzyć **postać kanoniczną problemu**:

$$ Minimalizuj_x\ c^T\mathbf{x} $$
$$ \textrm{z ograniczeniami } \mathbf{Ax} \le b $$
$$ \textrm{gdzie } \mathbf{x} \ge 0 $$

## Optymalizacja - problemy binarne

* W wielu przypadkach, probelm, który musimy rozwiązać przyjmuje postać zero-jedynkową: decyzji tak, albo nie.
* Wykorzystanie lub brak wykorzystnia określonego elementu, obecność lub brak obecności jakiejś składowej


**Przykład:**

* Firma ma kilka projektów, które można zlecić do relizacji
* Każdy projekt generuje określone zyski po jego zakończeniu
* Każdy projekt generuje pewne koszty w czasie swojego trwania

Pytanie: które projekty przyjąć do realizacji, a które odrzucić?

Tabela poniżej zawiera zestawienie projektów. 
* Wartości w poszczególnych miesiącach oznaczają koszt. 
* W kolumnie budżet wskazano środki, jakimi dysponuje firma w danym miesiącu - jest to nieprzekraczany budżet
* Wiersz **zysk** wskazuje na ostateczny zysk, który osiągnie firma po zakończeniu proejktu

| Projekt | 1  | 2 | 3 | 4 | Budżet |
|---|---|---|---|---|---|
| Styczeń  | 60  | 40 | 30  | 20  | 120 |
| Luty  | 20  | 30  | 10  | 20  | 85 |
| Marzec  | 45  | 20  | 20  | 30  | 100 |
| Zysk   | 220 | 120 | 90 | 110 | |

**Zmienne** 

Fakt wyboru / braku wyboru projektu można przeddsatwić jako zmienną binarną (1 lub zero). W notacji formalnej:

$$X_i = 1 \textrm{ jeśli projekt } i \textrm{ został wybrany, } 0 \textrm{ w przeciwnym wypadku}, \qquad \forall i \in \{1,2,3,4\}.$$

Ponieważ są 4 projekty, możemy je przedstawić jako wektor binary z czterema elementami

In [2]:
# number of projects
n = 4

# variables
x = cp.Variable((n, 1), boolean=True)

**Funkcja celu**


W optymalizacji kluczowe jest zdefinioawnie funkcji celu, tzn. tego, co próbujemy optymalizować. Przyjmuje ona postać równania liniowego. 

W przypadku opisywanego problemu - będzie to zysk po wykonaniu projektu. W tym przypadku mamy do czynienia z **maksymalizacją** zysku

$$\begin{aligned} &\textrm{Maximize } && 220X_1 + 120X_2 + 90X_3 + 110X_4 \end{aligned} $$

In [5]:
revenue = [220, 120, 90, 110]
objective = cp.Maximize(cp.sum(revenue * x))

C:\Users\filip\Anaconda3\lib\site-packages\cvxpy\expressions\expression.py:516: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)


**Ograniczenia**

Ograniczeniami w projekcie jest budżet dostępny w każdym miesiącu. 

Fakt uruchomienia / braku uruchomienia projektu, skutkuje poniesieniem określonych kosztów.

Łączna suma kosztów, nie może przekroczyć całości dostępnego budżetu.

$$\begin{aligned} & 60X_1 + 40X_2 + 30X_3 + 20 X_4 \leq 120 && \textrm{(Styczeń)} \\ & 20X_1 + 30X_2 + 10X_3 + 20X_4 \leq 85 && \textrm{(Luty)} \\ & 45X_1 + 20X_2 + 20X_3 + 30X_4 \leq 100 && \textrm{(Marzec)} \end{aligned} $$

co można przedstawić w postaci macierzowej:

$$\begin{aligned} & \left[\begin{array}{rrrr}60 & 40 & 30 & 20 \\ 20 & 30 & 10 & 20 \\ 45 & 20 & 20 & 30\end{array}\right]\left[\begin{array}{r}x_1\\x_2\\ x_3\\ x_4\end{array}\right] \leq \left[\begin{array}{r}120\\ 85 \\ 100\end{array}\right] \end{aligned}$$


In [6]:
project_cost = np.array([
    [60, 40, 30, 20],
    [20, 30, 10, 20],
    [45, 20, 20, 30]
])

total_cost = np.array([
    [120], 
    [85],
    [100]
])

constraints = [project_cost * x <= total_cost]

C:\Users\filip\Anaconda3\lib\site-packages\cvxpy\expressions\expression.py:516: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)


**Zdefiniowanie całości problemu**

Mając dane:
1. Zmienne
2. Funkcję celu
3. Ograniczenia

Można sformułować całe zadanie w postaci problemu optymalizacyjnego

$$\begin{aligned} &\textrm{Maksymalizuj } && \left[220, 120, 90, 119\right] \mathbf{x} \\ & \textrm{z ograniczeniami} && \left[\begin{array}{rrrr}60 & 40 & 30 & 20 \\ 20 & 30 & 10 & 20 \\ 45 & 20 & 20 & 30\end{array}\right]\mathbf{x} \leq \left[\begin{array}{r}120\\ 85 \\ 100\end{array}\right] \end{aligned}$$

In [7]:
# define the problem
problem = cp.Problem(objective, constraints)

**Rozwiązanie**

Rozwiązanie problemu uzyskiwane jest przez system optymalizacyjny, który dokonuje tego z użyciem najleszej strategii (dobieranej pod kątem typu problemu, wielkości poszczególnych macierzy, itp.).

System optymalizacyjny zwraca na wyjściu:

1. Status rozwiązania - czy udało się uzyskać wynik (czy problem w ogóle jest rozwiązywalny)
2. Wartość funkcji celu
3. Wartości zmiennych

In [8]:
# Rozpoczęcie rozwiązywania
problem.solve()

450.0

In [9]:
# Status rozwiązania
problem.status

'optimal'

In [10]:
# Wartości zmiennych
print(x.value)

[[1.]
 [1.]
 [0.]
 [1.]]


## Optymalizacja - problemy całkowitoliczbowe

Problemy całkowitoliczbową postać problemów optymalizacyjnych stosuje się wszędzie tam, gdzie składowych problemu nie da się podzeilić, tzn. muszą stanowić one jedną całość.

**Formalizacja**

Co do zasady, ich formalizacja wygląda identycznie jak w przypadku pozostałych problemów optymalizacyjnych.

Jedyną modyfikacją jest dodanie ograniczenia, iż zmienne muszą należeć do dziedziny liczb całkowitych:

$$ \mathbf{x} \in \mathbb{Z} $$ 

**Przykład: problem plecakowy**

Problem plecakowy jest jednym z klasycznych zagadnień podczas nauki programowania dynamicznego oraz optymalizacji.


Wyobraźmy sobie **złodzieja, który włamał się do muzeum**. Ma relatywnie mały plecak, do którego zmieści się **kilka przedmiotów**. W muzeum jest barzdo dużo eksponatów o różnej wielkości. Jak zdecydować, które z nich wybrać, tak aby **zmaksymalizować zysk ze sprzedaży na czarnym rynku?**

W muzeum znajdują sę nastęujące przedmioty, z których każdy występuje **w kilku egzemplarzach, tzn. można zabrać więcej niż 1 sztukę danego rodzaju**

1. **Figurka ze strożytnego Egiptu**, wielkość 3, wartość 400
2. **Popiersie z czasów Rzymskich**, wielkośc 4, wartość 500
3. **Miniaturowy portret z XIVw.**, wielkość 7, wartość 1000
4. **Fragment zbroi ze zdobieniami**, wielkość 8, wartość 1100
5. **Gobelin**, wielkość 9, wartość 1300

Plecak złodzieja ma **pojemność 17**

**Formalizacja**

Wektor **x** będzie przedstawiać ilośc obiektów danego typu, zabranych z muzeum (odpowiednio: figurek, popiersi, portretów, fragm. zbroi i gobelinów).

Element $x_i$ musi spełniać następujące warunki:

$$ x_i \ge 0 \textrm{ - musi być nieujemny} $$

$$ x_i \in \mathbb{Z} \textrm{ - musi należeć do zbioru liczb całkowitych} $$



In [25]:
x = cp.Variable(5, integer=True)
capacity = 17

**Zmienne**

Do ilości obiektów mamy przypisane odpowiednie wartości ze sprzedaży $v$ oraz wielkość $w$

In [23]:
w = [3, 4, 7, 8, 9]
v = [400, 500, 1000, 1100, 1300]

**Ograniczenia**

Złodziej nie może zabrać więcej niż zmieści się w jego plecaku, a więc:

$$ \mathbf{x}^Tw \le 17 $$

In [26]:
constraints = [cp.matmul(x, w) <= capacity]

**Funkcja celu**

Celem złodzieja jest zmaksymalizować wartość sprzedaży zrabowanych towarów, a zatem:

$$ Maksymalizuj_x\ \mathbf{x}^Tv $$

In [40]:
objective = cp.Maximize(cp.matmul(x, v))

**Definicja problemu**

Cały problem zostaje zatem przedstawiony jako:

$$ \begin{aligned} & v = [400, 500, 1000, 1100, 1300] && \textrm{ - wartości} \\ &  w = [3, 4, 7, 8, 9] && \textrm{ - wagi} \\ \\ & Maksymalizuj_x && \mathbf{x}^Tv  \\ & \textrm{z ograniczeniami } && \mathbf{x}^Tw \le 17 \\ & \textrm{gdzie } && \mathbf{x} \ge 0\ \wedge \mathbf{x} \in \mathbb{Z} \end{aligned} $$

In [36]:
x = cp.Variable(5, integer=True)
capacity = 17

w = [3, 4, 7, 8, 9]
v = [400, 500, 1000, 1100, 1300]

constraints = [
    cp.matmul(x, w) <= capacity, 
    x>=0]
objective = cp.Maximize(cp.matmul(x, v))
problem = cp.Problem(objective, constraints)

In [37]:
# Rozwiązanie

problem.solve()

2400.0

In [39]:
x.value

array([-0., -0., -0.,  1.,  1.])